In [242]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Example 4. PMF (Passenger Mix Flow) model

In [243]:
# Define the inputs

# L: set of flights
flights = pd.read_excel('Example_Data_PMF.xlsx', sheet_name='Flight').set_index('Flight Number')

# P: set of passenger itineraries
paths = pd.read_excel('Example_Data_PMF.xlsx', sheet_name='Itinerary').set_index('Itin No.')
paths['Leg 2'] = paths['Leg 2'].astype('Int64')

# P_p: set of passenger itineraries with recapture rate from itinerary p
P_p = pd.read_excel('Example_Data_PMF.xlsx', sheet_name='Recapture Rate').set_index(['p','r'])

# make a dictionary with the itinerary as the key and the rest as a sub-dictionary
paths = paths.to_dict(orient='index')
flights = flights.to_dict(orient='index')
P_p = P_p.to_dict(orient='index')

path_list = list(paths.keys())
flight_list = list(flights.keys())

# For all paths if 'Leg 1' and 'Leg 2' are numbers then create a list with both legs else, drop the keys from the list, and create a new key called 'Legs'
# else just change the name of the key 'Leg 1' to 'Legs'
for key in path_list:
    legs = []
    if type(paths[key]['Leg 1']) == int and type(paths[key]['Leg 2']) == int:
        legs.append(paths[key]['Leg 1'])
        legs.append(paths[key]['Leg 2'])
        paths[key]['Legs'] = legs
    elif type(paths[key]['Leg 1']) == int:
        legs.append(paths[key]['Leg 1'])
        paths[key]['Legs'] = legs
    del paths[key]['Leg 1']
    del paths[key]['Leg 2']


In [244]:
# Define path 999 with a fare of 0 and a demand of 0 
paths[999] = {'Legs': [], 'Demand': 0, 'Fare': 0}

In [245]:
# s_ip: binary variable indicating whether flight i is in itinerary p
s_ip = {}
for i in flight_list:
    for p in paths:
        s_ip[i,999] = 0
        if i in paths[p]['Legs']:
            s_ip[i,p] = 1
        else:
            s_ip[i,p] = 0

# Q_i: unconstrained demand for flight i = sum s_ip * demand of itinerary p for p in P
Q_i = {}
for i in flight_list:
    Q_i[i] = 0
    for p in paths:
        Q_i[i] += s_ip[i,p] * paths[p]['Demand']

# ds_i demand spill for flight i = Q_i - capacity of flight i
ds_i = {}
for i in flight_list:
    ds_i[i] = Q_i[i] - flights[i]['Capacity']

In [246]:
s_ip

{(102, 999): 0,
 (102, 1): 1,
 (102, 2): 0,
 (102, 3): 0,
 (102, 4): 0,
 (102, 5): 0,
 (102, 6): 0,
 (102, 7): 1,
 (102, 8): 0,
 (102, 9): 0,
 (102, 10): 0,
 (102, 11): 0,
 (102, 12): 0,
 (102, 13): 0,
 (102, 14): 0,
 (102, 15): 0,
 (301, 999): 0,
 (301, 1): 0,
 (301, 2): 0,
 (301, 3): 0,
 (301, 4): 0,
 (301, 5): 0,
 (301, 6): 0,
 (301, 7): 0,
 (301, 8): 0,
 (301, 9): 0,
 (301, 10): 1,
 (301, 11): 0,
 (301, 12): 0,
 (301, 13): 0,
 (301, 14): 1,
 (301, 15): 0,
 (201, 999): 0,
 (201, 1): 0,
 (201, 2): 0,
 (201, 3): 0,
 (201, 4): 0,
 (201, 5): 1,
 (201, 6): 0,
 (201, 7): 1,
 (201, 8): 0,
 (201, 9): 0,
 (201, 10): 0,
 (201, 11): 1,
 (201, 12): 0,
 (201, 13): 0,
 (201, 14): 0,
 (201, 15): 0,
 (104, 999): 0,
 (104, 1): 0,
 (104, 2): 1,
 (104, 3): 0,
 (104, 4): 0,
 (104, 5): 0,
 (104, 6): 0,
 (104, 7): 0,
 (104, 8): 1,
 (104, 9): 0,
 (104, 10): 0,
 (104, 11): 0,
 (104, 12): 0,
 (104, 13): 0,
 (104, 14): 0,
 (104, 15): 0,
 (202, 999): 0,
 (202, 1): 0,
 (202, 2): 0,
 (202, 3): 1,
 (202, 4): 0,


In [247]:
# Add entries to P_p for path 0 with a recapture rate of 1
for p in paths:
    P_p[p,999] = {'Recapture rate': 1}
    P_p[999,p] = {'Recapture rate': 0}

path_list = list(paths.keys())
flight_list = list(flights.keys())

P_p

{(1, 2): {'Recapture rate': 0.3},
 (2, 1): {'Recapture rate': 0.1},
 (3, 4): {'Recapture rate': 0.2},
 (4, 5): {'Recapture rate': 0.2},
 (3, 5): {'Recapture rate': 0.1},
 (6, 7): {'Recapture rate': 0.3},
 (7, 8): {'Recapture rate': 0.2},
 (6, 8): {'Recapture rate': 0.1},
 (9, 10): {'Recapture rate': 0.2},
 (10, 9): {'Recapture rate': 0.1},
 (11, 12): {'Recapture rate': 0.4},
 (12, 13): {'Recapture rate': 0.3},
 (11, 13): {'Recapture rate': 0.2},
 (12, 11): {'Recapture rate': 0.1},
 (14, 15): {'Recapture rate': 0.2},
 (15, 14): {'Recapture rate': 0.1},
 (4, 3): {'Recapture rate': 0.1},
 (1, 999): {'Recapture rate': 1},
 (999, 1): {'Recapture rate': 0},
 (2, 999): {'Recapture rate': 1},
 (999, 2): {'Recapture rate': 0},
 (3, 999): {'Recapture rate': 1},
 (999, 3): {'Recapture rate': 0},
 (4, 999): {'Recapture rate': 1},
 (999, 4): {'Recapture rate': 0},
 (5, 999): {'Recapture rate': 1},
 (999, 5): {'Recapture rate': 0},
 (6, 999): {'Recapture rate': 1},
 (999, 6): {'Recapture rate': 0},


In [248]:
# Implementation using column generation algorithm (CGA)

# Define the restricted master problem (RMP)
# All the spillage is reallocated to path 0

# Define the model
m = gp.Model('PMF')

# Define the decision variables
# t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
t_pr = m.addVars(path_list, name='t') 



In [250]:
# Define the objective function
# MINIMIZE (double sum of fare_p - bpr * fare_r) * t_pr
of = gp.quicksum((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t_pr[p] for r in [999] for p in path_list)
m.setObjective(of, GRB.MINIMIZE)

# Define the constraints
# Constraint 1: sum sum s_ip * t_pr - sum sum s_ip * brp * t_rp >= ds_i for all i but for r = 0 
m.addConstrs((gp.quicksum(s_ip[i,p] * t_pr[p] for p in path_list) - 
              gp.quicksum(s_ip[i,p] * P_p[(r,p)]['Recapture rate'] * t_pr[p] for p in path_list for r in [999]) >= 
              ds_i[i] for i in flight_list), name='π')

# Constraint 2: sum t_pr <= Dp for all p
m.addConstrs((t_pr[p] <= paths[p]['Demand'] for p in path_list), name='σ')

# Constraint 3: sum t_pr >= 0 for all p
m.addConstrs((t_pr[p] >= 0 for p in path_list), name='c3')

m.update()

In [251]:
# Solve the model
m.optimize()


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 42 rows, 16 columns and 52 nonzeros
Model fingerprint: 0x71139809
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 2e+02]
Presolve removed 39 rows and 11 columns
Presolve time: 0.01s
Presolved: 3 rows, 5 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8055000e+04   1.162500e+01   0.000000e+00      0s
       2    4.6580000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.658000000e+04


In [252]:
# Print the optimal objective value and the decision variables t_pr and the dual variables
print('Optimal objective value: %g' % m.objVal)
print('Optimal solution:')
for v in m.getVars():
    if v.x > 0:
        print('%s = %g' % (v.varName, v.x))

print('Dual variables:')
for c in m.getConstrs():
    if c.Pi != 0:
        print('%s = %g' % (c.ConstrName, c.Pi))

# Save dual variables in a dictionary
pi_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'π':
        pi_dual[c.ConstrName] = c.Pi

sigma_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'σ':
        sigma_dual[c.ConstrName] = c.Pi
        

Optimal objective value: 46580
Optimal solution:
t[2] = 12
t[5] = 50
t[6] = 12
t[7] = 62
t[8] = 70
t[9] = 32
t[10] = 87
t[12] = 72
t[13] = 30
Dual variables:
π[102] = 100
π[104] = 170
π[202] = 80
π[302] = 140
π[203] = 180
π[101] = 120
σ[5] = -90
σ[8] = -250
σ[13] = -100


In [255]:
# Create the pricing problem (PP)

tpr_prime = {}
# tpr = (fare_p - sum (π_i) for i being each flight in path p) - bpr * (fare_r - sum (π_j) for j being each flight in path p)) - σ_p
for p,r in P_p.keys():
    t_prime_pr = ((paths[p]['Fare'] - sum(pi_dual['π[' + str(i) + ']'] for i in paths[p]['Legs'])) -
                      (P_p[(p,r)]['Recapture rate']) *
                      (paths[r]['Fare'] - sum(pi_dual['π[' + str(j) + ']'] for j in paths[r]['Legs'])) -
                      (sigma_dual['σ[' + str(p) + ']']))
    if t_prime_pr < 0:
        tpr_prime[p,r] = t_prime_pr
    
tpr_prime

{(2, 1): -5.0, (3, 4): -14.0, (12, 11): -15.0}

In [ ]:
# Add the new columns to the RMP
for p,r in tpr_prime.keys():
    t_pr[p] = m.addVar(obj=tpr_prime[p,r], name='t_p' + str(p) + str(r))
    m.update()

# Solve the model